In [0]:
from google.colab import files
files.upload()
# !mkdir Images Annotations model
# !mv *.jpg Images
# !mv *.xml Annotations
# !pip install shapely

In [0]:
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
import os
from scipy.signal import medfilt
import skimage.io
import skimage.color
from itertools import cycle
from sklearn import metrics
from random import shuffle
from matplotlib.path import Path
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
%matplotlib inline

В папке Annotations хранятся xml файлы с разметкой, а в папке Images - ионограммы.
Напишем функции для извлечения полигонов с разметкой и проверки принадлежности пикселя к определенному полигону.

In [0]:
def labeling(name):    
    tree = ET.parse(r"Annotations/"+name[:-4]+".xml")
    root = tree.getroot()
    f_layer_points = []
    e_layer_points = []
    for child in root.findall('object'):
        if child.find('name').text == 'f-layer':
            for points in child.find("polygon").findall("pt"):
                f_layer_points.append((points[0].text,points[1].text))
        elif child.find('name').text == 'e-layer':
            for points in child.find("polygon").findall("pt"):
                e_layer_points.append((points[0].text,points[1].text))
    return np.array(f_layer_points,dtype = 'float32'),np.array(e_layer_points,dtype = 'float32')

In [0]:
def point_in_polygon(coords,f_layer,e_layer):
    try:
        polygon_f = Polygon(np.array(f_layer,dtype='float32'))
    except AssertionError:
        f_layers = [(0.,0.),(1.,0.),(0.,1.)]#оказывается слой f тоже, видимо забыл
        polygon_f = Polygon(f_layers)
    try:
        polygon_e = Polygon(np.array(e_layer,dtype='float32'))#на некоторых ионограммах слой E не отмечен
    except AssertionError:                                    #следовательно e_layer был пуст
        e_layers = [(0.,0.),(1.,0.),(0.,1.)]
        polygon_e = Polygon(e_layers)
    point = Point(coords)
    if polygon_f.contains(point) == True:
        return [0,1]
    elif polygon_e.contains(point) == True:
        return [0,1]
    else:
        return [1,0]

In [0]:
def padding(image):#функция для прибавки нулей вдоль границы изображения
    new_image = np.zeros((image.shape[0]+80,image.shape[1]+80))
    new_image[40:40+image.shape[0],40:40+image.shape[1]] = image
    return new_image

In [0]:
def batch(image,i,j):#формат изображения 329х240
    image1 = image[i:i+81,j:j+81]#выделяет кусочек изображения для подачи на вход сети
    return image1

Теперь напишем код для сверточной нейронной сети. Для сегментации изображения используем Patch-based подход. Сеть будет обучаться на изображениях размера 81х81. Ниже приведены функции, используемые для удобства при написании кода нейронной сети.

In [0]:
def bias(shape):#смещения
    return tf.Variable(tf.constant(0.1,shape=shape))

def weight(shape):#веса
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def conv2d(x,W):#операция свертки
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool(h):# операция выбора с усреднением
    return tf.nn.max_pool(h,ksize = [1,2,2,1],strides = [1,2,2,1],padding='SAME')

In [0]:
with tf.device('/gpu:0'):    #основой сети послужил фрагмент из туториала tensorflow
  x = tf.placeholder(tf.float32,shape = [None,81,81],name='x')
  y_ = tf.placeholder(tf.float32,shape = [None,2],name='y_')
  x_image = tf.reshape(x,[-1,81,81,1],name='x_image')

  weight_1 = weight([4,4,1,40])
  bias_1 = bias([40])
  h_conv1 = tf.nn.relu(conv2d(x_image,weight_1)+bias_1,name='h_conv1')
  h_pool1 = max_pool(h_conv1)


  weight_2 = weight([5,5,40,40])
  bias_2 = bias([40])
  h_conv2 = tf.nn.relu(conv2d(h_pool1,weight_2)+bias_2,name='h_conv2')
  h_pool2 = max_pool(h_conv2)

  weight_3 = weight([4,4,40,40])
  bias_3 = bias([40])
  h_conv3 = tf.nn.relu(conv2d(h_pool2,weight_3)+bias_3,name='h_conv3')
  h_pool3 = max_pool(h_conv3)


  w_fc1 = weight([11*11*40,100])
  b_fc1 = bias([100])
  h_pool4_flat = tf.reshape(h_pool3,[-1,11*11*40])
  h_fc1 = tf.nn.relu(tf.matmul(h_pool4_flat,w_fc1)+b_fc1,name='h_fc1')


  w_fc2 = weight([100,2])
  b_fc2 = bias([2])
  h_fc2 = tf.matmul(h_fc1,w_fc2)+b_fc2

  softmax = tf.nn.softmax(h_fc2,name='softmax')

  cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=h_fc2),name='cross_entropy')#функционал ошибки
  train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)#оптимизатор

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ionograms = os.listdir(r"Images/")
    shuffle(ionograms)#считываем все файлы из директории и перемешиваем
    train_set = ionograms[0:801]
    test_set = ionograms[801:]#тренировочный и тестовый датасеты
    pool = cycle(train_set)#создаем цикличный список из файлов
    saver = tf.train.Saver()
    for k in range(85):
        print("Step: {}".format(k+1))
        ionogram = next(pool)
        image = skimage.io.imread(r"Images/"+ionogram)[:,:,1]*(-1)
#         image = skimage.io.imread(r"Images/"+ionogram)
        f_layer,e_layer = labeling(ionogram)
        try:
          f_layer[:,1] = 238-f_layer[:,1]#меняем систему координат
        except IndexError:
          continue
        if len(e_layer)!=0:
          e_layer[:,1] = 238-e_layer[:,1]
        padded_image = padding(image)#добавляем нулей с краёв изображения
        background = []
        layer = []
        for i in range(image.shape[0]):#добавляем координаты точек из фона и из слоев
            for j in range(image.shape[1]):
                  if point_in_polygon((j,238-i),f_layer,e_layer)[0]==1:
                    background.append((i,j))#из фона
                  elif point_in_polygon((j,238-i),f_layer,e_layer)[1]==1:
                    layer.append((i,j))#из слоя
        relation = len(background)//len(layer)#отношение количества пикселей из слоя и пикселей из фона(для балансировки выборки)
        print(len(layer),len(background))
        for index in range(len(layer)):
          labels = []
          images_train = []
          images_train.append(batch(padded_image,*layer[index]))
          images_train.append(batch(padded_image,*background[index*relation]))
          layer_coords = layer[index][1],238-layer[index][0]
          background_coords = background[index*relation][1],238-background[index*relation][0]
          labels.append(point_in_polygon(layer_coords,f_layer,e_layer))
          labels.append(point_in_polygon(background_coords,f_layer,e_layer))
          train_step.run(feed_dict={x:images_train,y_:labels})
        saver.save(sess,r'model/diploma',global_step = k)#сохраняем модель для дальнейшего использования

In [0]:
sess2 = tf.Session()
saver = tf.train.import_meta_graph('model/diploma-84.meta')#восстанавливаем модель
saver.restore(sess2,r'model/diploma-84')
image = skimage.io.imread(r"Images/"+test_set[6])[:,:,1]*(-1)
padded_image = np.array(padding(image),dtype='float32')
result = np.zeros((image.shape[0],image.shape[1]))
f_layer,e_layer = labeling(train_set[i])
y_pred = []
y_true = []
for i in range(image.shape[0]):
    print("Step: ",i)
    for j in range(image.shape[1]):
        label = point_in_polygon((j,239-i),f_layer,e_layer)
        feed_dict = {x: np.reshape(padded_image[i:i+81,j:j+81],(1,81,81))}
        prediction = sess2.run(softmax,feed_dict)
        #print(prediction)
        if np.argwhere(prediction[0]==np.max(prediction[0]))==0:
          result[i,j] = 0
        elif np.argwhere(prediction[0]==np.max(prediction[0]))==1 & (prediction[0,1]<0.8)==True:
          result[i,j] = 0
        else:
          result[i,j] = 1
        y_pred.append(result[i,j])
        y_true.append(label[0])
# print(len(y_pred),len(y_true))
prec = metrics.precision_score(y_true,y_pred)#считаем точность
rec = metrics.recall_score(y_true,y_pred)#считаем полноту(пока получается не очень)
print(prec,rec)
sess2.close()
skimage.io.imshow(result,cmap = plt.cm.gray)